In [53]:
import os
import numpy as np, seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy



#importing Heart  Data
Heart_Data = pd.read_csv(r'heart.csv')
Heart_Data

FileNotFoundError: [Errno 2] No such file or directory: 'heart.csv'

In [ ]:
#Initial Exploration
# Number of rows
print(Heart_Data.shape[0])

# Column names
col=Heart_Data.columns.tolist()
print(Heart_Data.columns.tolist())

# Data types
print(Heart_Data.dtypes)


In [ ]:
#Data Cleaning 
#Check if there are some null values
Heart_Data.isnull().sum().sort_values()
Heart_Data.shape

In [ ]:
# Create dummies variables
#thall column
Heart_Data['thall'].value_counts()
thall_dum=pd.get_dummies(Heart_Data['thall'])

thall_dum = thall_dum.set_axis(['thall_None', 'thall_Fixed_Defect', 'thall_Normal','thall_Reversable'], axis=1)
print(thall_dum)
#Slope column
Heart_Data['slp'].value_counts()
slope_dum=pd.get_dummies(Heart_Data['slp'])

slope_dum = slope_dum.set_axis(['slp_Downsloping', 'slp_Flat', 'slp_Upsloping'], axis=1)
print(slope_dum)
#Chest Pain column
Heart_Data['cp'].value_counts()
chest_pain_dum=pd.get_dummies(Heart_Data['cp'])

chest_pain_dum = chest_pain_dum.set_axis(['cp_asymptomatic', 'cp_typical_angina', 'cp_atypical_angina','cp_no_angina'], axis=1)
print(chest_pain_dum)
#Rest Ecg column
Heart_Data['restecg'].value_counts()
rest_ecg=pd.get_dummies(Heart_Data['restecg'])

rest_ecg = rest_ecg.set_axis(['restecg_hypetrophic', 'restecg_normal', 'restecg_abnormal'], axis=1)
print(rest_ecg)

In [ ]:
#Edit Data Frame
Heart_Data_bk=Heart_Data
Heart_Data=Heart_Data.drop(columns=['slp','thall','restecg','cp'])
target=Heart_Data['output']
Heart_Data= Heart_Data.join(rest_ecg)
Heart_Data= Heart_Data.join(chest_pain_dum)
Heart_Data= Heart_Data.join(thall_dum)
Heart_Data= Heart_Data.join(slope_dum)

In [ ]:
Heart_Data
y=Heart_Data['output']
fields = list(Heart_Data.drop(columns='output'))  # everything except "output"
correlations = Heart_Data[fields].corrwith(y)
correlations.sort_values(inplace=True)
correlations

In [ ]:
os.chdir(r"C:\Users\gdelprat\OneDrive - Capgemini\Desktop\Corsi\Coursera ML\Supervised_ML_Class\Data")

from colorsetup import colors, palette
sns.set_palette(palette)
sns.set_context('talk')
sns.set_palette(palette)
sns.set_style('white')



#ns.pairplot(Heart_Data)
sns.barplot(fields,correlations)

In [ ]:
#Numerical Data
Heart_Data.describe()

In [ ]:
# Create a functions to plot multiple bar charts
def hist_loop(data: pd.DataFrame,
              rows: int,
              cols: int,
              figsize: tuple):
    fig, axes = plt.subplots(rows,cols, figsize=figsize)
    for i, ax in enumerate(axes.flatten()):
        if i < len(data.columns):
            data[sorted(data.columns)[i]].plot.hist(bins=30, ax=ax)
            ax.set_title(f'{sorted(data.columns)[i]} distribution', fontsize=10)
            ax.tick_params(axis='x', labelsize=10)
            ax.tick_params(axis='y', labelsize=10)
            ax.get_yaxis().get_label().set_visible(False)
        else:
            fig.delaxes(ax=ax)
    fig.tight_layout()



In [ ]:
Heart_Data_num=Heart_Data_bk.drop(columns=['slp','thall','restecg','cp','sex','output','caa','exng','fbs'])
hist_loop(data=Heart_Data_num,
          rows=3,
          cols=3,
          figsize=(20,10))

In [ ]:
# Perform log transformation
for col in Heart_Data_num.columns.values:
    Heart_Data_num['log_' + col] = Heart_Data_num[col].apply(np.log1p)
Heart_Data_num

In [ ]:
log_df = Heart_Data_num.filter(regex='^log_', axis=1)
hist_loop(data=log_df,
          rows=3,
          cols=3,
          figsize=(20,10))


Heart_Data_num_log=Heart_Data_num.drop(columns=['age','chol','trtbps','thalachh','oldpeak'])

In [ ]:
#Join log data with categorical data
Heart_Data=Heart_Data.drop(columns=['age','chol','trtbps','thalachh','oldpeak'])
Heart_Data_Fin=Heart_Data.join(Heart_Data_num_log)
Heart_Data_Fin.to_csv('Heart_Data_Clean.csv',index='False')


In [ ]:
#Hypothesis Testing 
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind, t
import math

s1= Heart_Data_Fin['log_age']
s2= Heart_Data_Fin['log_chol']

result = ttest_ind(s1, s2, equal_var=False)
print("t-value" + " "+ str(result.statistic))
print("p-value" + " " + str(result.pvalue))

In [ ]:
s3= Heart_Data_Fin['log_age']
s4= Heart_Data_Fin['log_trtbps']

result2 = ttest_ind(s3, s4, equal_var=False)
print("t-value" + " "+ str(result2.statistic))
print("p-value" + " " + str(result2.pvalue))

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline


#Regression  
x=Heart_Data_Fin.drop(columns='output')
y=Heart_Data_Fin['output']
print(y)
#K-Fold
kf = KFold(shuffle=True, random_state=72018, n_splits=3)


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
#Linear Regression
# vanilla regression and K-fold cross validation
s = StandardScaler()
lr = LinearRegression()
scores = []
lr = LinearRegression()

for train_index, test_index in kf.split(x):
    X_train, X_test, y_train, y_test = (x.iloc[train_index, :], 
                                        x.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])
    
    lr.fit(X_train, y_train)
        
    y_pred = lr.predict(X_test)

    score = r2_score(y_test.values, y_pred)
    
    scores.append(score)
    
scores

# with pipeline
estimator = Pipeline([("scaler", s),("regression", lr)])
predictions_lr = cross_val_predict(estimator, X_train, y_train, cv=kf)
linear_score = r2_score(y_train, predictions_lr)

linear_score, score #almost identical

In [ ]:
#Adding Polynomial Tranformations

# lasso regression and K-fold cross validation
s = StandardScaler()
pf = PolynomialFeatures(degree=2)
kf = KFold(shuffle=True, random_state=72018, n_splits=3)
scores = []
alphas = np.arange(10e-2, 1, 0.1)
predictions_lsr = []
for alpha in alphas:
    las = Lasso(alpha=alpha, max_iter=100000)
    
    estimator = Pipeline([
        ("scaler", s),
        ("make_higher_degree", pf),
        ("lasso_regression", las)])

    predictions_lsr = cross_val_predict(estimator, X_train, y_train, cv = kf)
    
    score = r2_score(y_train, predictions_lsr)
    
    scores.append(score)
plt.semilogx(alphas, scores, '-o', color='purple')
plt.title('Lasso Regression')
plt.xlabel('$\\alpha$')
plt.ylabel('$R^2$');

In [ ]:
best_estimator = Pipeline([
                    ("scaler", s),
                    ("make_higher_degree", PolynomialFeatures(degree=2)),
                    ("lasso_regression", Lasso(alpha=0.03))])

best_estimator.fit(X_train, y_train)
lasso_score = best_estimator.score(X_train, y_train)
lasso_score

In [ ]:
# ridge regression and K-fold cross validation
pf = PolynomialFeatures(degree=2)
alphas = np.arange(10e-2, 1, 0.1)
scores=[]
predictions_rr = []
for alpha in alphas:
    ridge = Ridge(alpha=alpha, max_iter=100000)

    estimator = Pipeline([
        ("scaler", s),
        ("polynomial_features", pf),
        ("ridge_regression", ridge)])

    predictions_rr = cross_val_predict(estimator, X_train, y_train, cv = kf)
    score = r2_score(y_train, predictions_rr)
    scores.append(score)

plt.plot(alphas, scores, '-o', color='purple')
plt.title('Ridge Regression')
plt.xlabel('$\\alpha$')
plt.ylabel('$R^2$');

In [ ]:

best_estimator = Pipeline([
                    ("scaler", s),
                    ("make_higher_degree", PolynomialFeatures(degree=2)),
                    ("ridge_regression", Ridge(alpha=0.03))])

best_estimator.fit(X_train, y_train)
ridge_score = best_estimator.score(X_train, y_train)
# comparing scores
pd.DataFrame([[linear_score, lasso_score, ridge_score]],columns=['linear', 'lasso', 'ridge'], index=['score'])

In [ ]:
#Logistic Regression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
#Dataframe for metrics
metrics = pd.DataFrame()

# Standard logistic regression
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

precision_lr, recall_lr = (round(float(x),2) for x in list(score(y_test,
                                                                    y_pred_lr,
                                                                    average='weighted'))[:-2])
# writing all lr stats to metrics DataFrame
lr_stats = pd.Series({'precision':precision_lr,
                      'recall':recall_lr,
                      'accuracy':round(accuracy_score(y_test, y_pred_lr), 2),
                      'f1score':round(f1_score(y_test, y_pred_lr), 2),
                      'auc': round(roc_auc_score(y_test, y_pred_lr),2)},
                     name='Logistic Regression')
# Report outcomes
pd.DataFrame(classification_report(y_test, y_pred_lr, output_dict=True)).iloc[:3,:2]

In [ ]:
#K Nearest Neighbors model

knn=KNeighborsClassifier(n_neighbors=3 , weights='distance')
knn = knn.fit(X_train,y_train)
y_pred_knn=knn.predict(X_test)
precision_knn, recall_knn = (round(float(x),2) for x in list(score(y_test,y_pred_knn,average='weighted'))[:-2])
# adding KNN stats to metrics DataFrame
knn_stats = pd.Series({'precision':precision_knn,
                      'recall':recall_knn,
                      'accuracy':round(accuracy_score(y_test, y_pred_knn), 2),
                      'f1score':round(f1_score(y_test, y_pred_knn), 2),
                      'auc': round(roc_auc_score(y_test, y_pred_knn),2)}, name='KNN')
# Report outcomes
pd.DataFrame(classification_report(y_test, y_pred_knn, output_dict=True)).iloc[:3,:2]

In [ ]:

# Initialize the random forest estimator
RF = RandomForestClassifier(oob_score=True, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)

# initialise list for out of bag error
oob_list = list()

# Iterate through all of the possibilities for number of trees
for n_trees in [15, 20, 30, 40, 50, 100, 150, 200, 300, 400, 500 ,600 , 700, 800 , 900,1000,1200,1900]:
    
    # Use this to set the number of trees
    RF.set_params(n_estimators=n_trees)
    
    # Fit the model
    RF.fit(X_train, y_train)
    
    # Get the out of bag error and store it
    oob_error = 1 - RF.oob_score_
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

rf_oob_df = pd.concat(oob_list, axis=1).T.set_index('n_trees')

In [ ]:
sns.set_context('talk')
sns.set_style('white')

ax = rf_oob_df.plot(legend=False, marker='o', color="green", figsize=(14, 7), linewidth=5)
ax.set(ylabel='out-of-bag error');

In [ ]:
rf = RF.set_params(n_estimators=200)

y_pred_rf = rf.predict(X_test)
precision_rf, recall_rf = (round(float(x),2) for x in list(score(y_test,
                                                                    y_pred_rf,
                                                                    average='weighted'))[:-2])
rf_stats = pd.Series({'precision':precision_rf,
                      'recall':recall_rf,
                      'accuracy':round(accuracy_score(y_test, y_pred_rf), 2),
                      'f1score':round(f1_score(y_test, y_pred_rf), 2),
                      'auc': round(roc_auc_score(y_test, y_pred_rf),2)}, name='Random Forest')
# Report outcomes
pd.DataFrame(classification_report(y_test, y_pred_rf, output_dict=True)).iloc[:3,:2]

In [ ]:
#Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)
dt.tree_.node_count, dt.tree_.max_depth

In [ ]:

y_train_pred = dt.predict(X_train)
y_pred_dt = dt.predict(X_test)

precision_dt, recall_dt = (round(float(x),2) for x in list(score(y_test,
                                                                y_pred_dt,
                                                                average='weighted'))[:-2])
# adding dt stats to metrics DataFrame
dt_stats = pd.Series({'precision':precision_dt,
                      'recall':recall_dt,
                      'accuracy':round(accuracy_score(y_test, y_pred_dt), 2),
                      'f1score':round(f1_score(y_test, y_pred_dt), 2),
                      'auc': round(roc_auc_score(y_test, y_pred_dt),2)}, name='Decision Tree')
# Report outcomes
pd.DataFrame(classification_report(y_test, y_pred_dt, output_dict=True)).iloc[:3,:2]

In [ ]:
fig, axList = plt.subplots(nrows=2, ncols=2)
axList = axList.flatten()
fig.set_size_inches(12, 10)


models = coeff_labels = ['lr', 'knn', 'dt', 'rf']
cm = [confusion_matrix(y_test, y_pred_lr),
      confusion_matrix(y_test, y_pred_knn),
      confusion_matrix(y_test, y_pred_dt),
      confusion_matrix(y_test, y_pred_rf)]
labels = ['False', 'True']

for ax,model, idx in zip(axList, models, range(0,4)):
    sns.heatmap(cm[idx], ax=ax, annot=True, fmt='d', cmap='winter');
    ax.set(title=model);
    ax.set_xticklabels(labels, fontsize=20);
    ax.set_yticklabels(labels, fontsize=20);
    ax.set_ylabel('Prediction', fontsize=25);
    ax.set_xlabel('Ground Truth', fontsize=25)
    
plt.tight_layout()

In [ ]:
metrics.append([lr_stats, knn_stats, dt_stats, rf_stats])

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

